# Imports & Data Loading

In [1]:
import numpy as np
import pandas as pd

from functools import reduce


import plotly.express as px
import plotly.graph_objects as go



import warnings
warnings.filterwarnings('ignore')

In [2]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


In [3]:
dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung']
    
    
    

## Data Processing

In [13]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



In [22]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)
globe_df = globe_df.loc[[142]]


In [23]:
globe_df.to_csv('../upload_data/test_globe.csv')

In [37]:
test_globe_df = pd.read_csv('../upload_data/test_globe.csv')

In [38]:
test_globe_df

,Unnamed: 0,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,amount_2014,clim_rel_amount_2014,clim_adapt_amount_2014,clim_miti_amount_2014,amount_2015,...,clim_adapt_amount_2020,clim_miti_amount_2020,amount_2021,clim_rel_amount_2021,clim_adapt_amount_2021,clim_miti_amount_2021,amount_2022,clim_rel_amount_2022,clim_adapt_amount_2022,clim_miti_amount_2022
0,142,5627.252576,992.115751,430.780688,561.335063,6872.547503,1197.406245,599.863973,597.542272,5880.075415,...,1509.883531,1054.035251,11116.566037,2972.043199,1596.187821,1376.035353,12068.758817,3333.997495,1693.406647,1640.85655


In [39]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    test_globe_df[non_clim_col] = test_globe_df[amount_col] - (test_globe_df[clim_adapt_col] + test_globe_df[clim_miti_col])


In [40]:
test_globe_df

,Unnamed: 0,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,amount_2014,clim_rel_amount_2014,clim_adapt_amount_2014,clim_miti_amount_2014,amount_2015,...,non_clim_amount_2013,non_clim_amount_2014,non_clim_amount_2015,non_clim_amount_2016,non_clim_amount_2017,non_clim_amount_2018,non_clim_amount_2019,non_clim_amount_2020,non_clim_amount_2021,non_clim_amount_2022
0,142,5627.252576,992.115751,430.780688,561.335063,6872.547503,1197.406245,599.863973,597.542272,5880.075415,...,4635.136825,5675.141257,4727.342099,5715.66272,5955.285504,6593.216318,6815.3481,8255.063184,8144.342864,8734.49562


In [42]:
# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


In [43]:
melted_df

,Type_Year,Amount
0,clim_adapt_amount_2013,430.780688
1,clim_adapt_amount_2014,599.863973
2,clim_adapt_amount_2015,661.362943
3,clim_adapt_amount_2016,893.345489
4,clim_adapt_amount_2017,865.653914
5,clim_adapt_amount_2018,1048.374900
6,clim_adapt_amount_2019,1242.999296
7,clim_adapt_amount_2020,1509.883531
8,clim_adapt_amount_2021,1596.187821
9,clim_adapt_amount_2022,1693.406647


In [44]:
# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amount'] * 1_000_000




In [55]:
melted_df.to_csv("../upload_data/split_globe.csv")

In [56]:
read_df = pd.read_csv('../upload_data/split_globe.csv')
read_df.head(2)

,Unnamed: 0,Type_Year,Amount,Year,Type
0,0,clim_adapt_amount_2013,430780688.5,2013,Klimaanpassung Finanzierung
1,1,clim_adapt_amount_2014,599863973.0,2014,Klimaanpassung Finanzierung


In [51]:
read_df['Type'].unique()

array(['Klimaanpassung Finanzierung', 'Klimaschutz Finanzierung',
       'Andere ODA'], dtype=object)

In [58]:
split_df = pd.read_csv('../upload_data/split_globe.csv')


fig = px.bar(split_df, x='Year', y='Amount', color='Type',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Type': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

In [9]:
dictionary_df['2015'][['Sector', 'SECTOR']]

,Sector,SECTOR
0,IV.1. General Environment Protection,41010
1,IV.1. General Environment Protection,41020
2,IV.2. Other Multisector,43010
3,IV.2. Other Multisector,43010
4,IV.2. Other Multisector,43030
...,...,...
12596,I.1.b. Basic Education,11230
12598,IV.2. Other Multisector,43010
12601,IV.2. Other Multisector,43010
12611,I.5.a. Government & Civil Society-general,15110
